<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_negative_energy_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_negative_energy_ai.py

Full pipeline for NegativeEnergyAI:
  1. Synthetic dataset for Casimir force: (distance, permittivity) → force
  2. Physics-informed loss: match true Casimir force in log-space
  3. MLP with LayerNorm & Dropout for uncertainty
  4. MC-Dropout inference to quantify predictive variance
  5. Training loop with Adam, ReduceLROnPlateau, and early stopping
  6. Evaluation: scatter, surface plots, and uncertainty heatmap
"""

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import TensorDataset, DataLoader, random_split
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# ------------------------------------------------------------------------------
# 1. True Casimir Force (vectorized)
# ------------------------------------------------------------------------------
def casimir_force(d, eps):
    """
    Compute Casimir force for arrays of distances and permittivities.
    Negative by definition.
    """
    return - (np.pi**2) / (240.0 * d**4) * eps

# ------------------------------------------------------------------------------
# 2. Dataset: log-sampled distances, normalized for stability
# ------------------------------------------------------------------------------
class CasimirDataset(torch.utils.data.Dataset):
    def __init__(self, n_dist=200, n_eps=200):
        # distances in meters: 10nm → 1µm (log-uniform)
        d = np.exp(np.linspace(np.log(1e-8), np.log(1e-6), n_dist))
        e = np.linspace(1.0, 10.0, n_eps)
        D, E = np.meshgrid(d, e)
        X = np.stack([D.ravel(), E.ravel()], axis=1).astype(np.float32)
        y = casimir_force(X[:,0], X[:,1]).astype(np.float32).reshape(-1,1)

        # log-transform distance and force & normalize
        log_d = np.log(X[:,0])
        log_y = np.log(-y[:,0])  # log(|F|)

        self.X = torch.tensor(
            np.stack([
                (log_d - log_d.mean())/log_d.std(),
                (X[:,1] - X[:,1].mean())/X[:,1].std()
            ], axis=1),
            dtype=torch.float32
        )
        self.y = torch.tensor(
            ((log_y - log_y.mean())/log_y.std()).reshape(-1,1),
            dtype=torch.float32
        )
        self.orig = (D, E, y.reshape(D.shape))  # for plotting

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ------------------------------------------------------------------------------
# 3. Model: NegativeEnergyAI with Dropout & LayerNorm
# ------------------------------------------------------------------------------
class NegativeEnergyAI(nn.Module):
    def __init__(self, input_dim=2, hidden_dims=[64,64], output_dim=1, p_drop=0.1):
        super().__init__()
        layers = []
        dim = input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(dim, h),
                nn.LayerNorm(h),
                nn.ReLU(),
                nn.Dropout(p_drop)
            ]
            dim = h
        layers.append(nn.Linear(dim, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ------------------------------------------------------------------------------
# 4. Loss in log-space
# ------------------------------------------------------------------------------
def pinn_loss(pred, true):
    """
    MSE in normalized log-space of |F|
    """
    return nn.MSELoss()(pred, true)

# ------------------------------------------------------------------------------
# 5. MC-Dropout inference for uncertainty
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, x, n_samples=100):
    """
    Returns mean and std of model(x) in log-space.
    """
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(n_samples):
            preds.append(model(x).cpu().numpy())
    arr = np.stack(preds, axis=0)
    return arr.mean(axis=0), arr.std(axis=0)

# ------------------------------------------------------------------------------
# 6. Training and Evaluation
# ------------------------------------------------------------------------------
def main():
    # Hyperparams
    batch_size = 256
    lr         = 1e-3
    max_epochs = 200
    patience   = 15
    device     = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Data
    dataset = CasimirDataset(n_dist=200, n_eps=200)
    n_val    = int(0.2 * len(dataset))
    n_train  = len(dataset) - n_val
    train_ds, val_ds = random_split(dataset, [n_train, n_val])
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=batch_size)

    # Model
    model = NegativeEnergyAI().to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5
    )

    best_val = float('inf')
    wait      = 0
    history  = {'train': [], 'val': []}

    # Training loop
    for epoch in range(1, max_epochs+1):
        model.train()
        train_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred   = model(xb)
            loss   = pinn_loss(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * xb.size(0)

        train_loss /= len(train_loader.dataset)

        # validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred   = model(xb)
                loss   = pinn_loss(pred, yb)
                val_loss += loss.item() * xb.size(0)
        val_loss /= len(val_loader.dataset)

        scheduler.step(val_loss)
        history['train'].append(train_loss)
        history['val'].append(val_loss)

        print(f"Epoch {epoch:03d} | Train {train_loss:.4e} | Val {val_loss:.4e}")

        if val_loss < best_val:
            best_val = val_loss
            torch.save(model.state_dict(), "best_neg_energy.pth")
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print("Early stopping")
                break

    # load best
    model.load_state_dict(torch.load("best_neg_energy.pth"))

    # Evaluate on full grid
    D, E, F_true = dataset.orig
    X_full = dataset.X.to(device)
    mean_log_f, std_log_f = mc_dropout_predict(model, X_full, n_samples=200)
    mean_f = np.exp(mean_log_f * dataset.y.std().item() + dataset.y.mean().item())
    std_f  = std_log_f * dataset.y.std().item()  # approx error on log-space

    # Scatter plot
    F_mod = F_true.ravel()
    plt.figure(figsize=(6,6))
    plt.scatter(np.abs(F_mod), mean_f.ravel(), s=5, alpha=0.3)
    lims = [F_mod.min(), F_mod.max()]
    plt.plot(lims, lims, 'r--')
    plt.xscale('log'); plt.yscale('log')
    plt.xlabel("True |Casimir Force|")
    plt.ylabel("Predicted |Casimir Force|")
    plt.title("Log-Log Fit")
    plt.tight_layout()
    plt.show()

    # Heatmap of uncertainty
    plt.figure(figsize=(6,5))
    plt.pcolormesh(
        np.log10(D*1e6), E,
        std_f.reshape(D.shape).T,
        shading='auto', cmap='magma'
    )
    plt.colorbar(label="Std(log|F|)")
    plt.xlabel("log10(Distance [µm])")
    plt.ylabel("Permittivity")
    plt.title("Prediction Uncertainty")
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()